# Computer Science Studio 1 - Assignment 2

---



**Topic**: *Link Prediction Methods and Their Accuracy for Different Social Networks and Network Metrics*



In [0]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
from sklearn import metrics
from google.colab import drive
from datetime import datetime
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
drive.mount('/content/gdrive')

# 1. Import dataset

In [0]:
# Youtube
df = pd.read_csv('youtube_friendship.txt', sep=" ", names = ['u', 'v', 'useless', 'timestamp']).drop(columns = ['useless'])
df['timestamp'] = df['timestamp'].apply(lambda x: datetime.strptime(datetime.utcfromtimestamp(int(x)).strftime('%Y-%m-%d'), '%Y-%m-%d'))

In [0]:
df.timestamp.unique()

In [0]:
org_train = df[df['timestamp'] < datetime.strptime('2007-07-01', '%Y-%m-%d')].drop(columns = ['timestamp'])
org_test = df[df['timestamp'] >= datetime.strptime('2007-07-01', '%Y-%m-%d')].drop(columns = ['timestamp'])
del df

In [0]:
print("org_train shape " + str(org_train.shape))
print("org_test shape " + str(org_test.shape))

org_train shape (7710599, 2)
org_test shape (1664775, 2)


# 2. Preprocessing


In [0]:
def create_graph(df, un_graph = True): # returns nx.graph or nx.DiGraph
  """
   Create a graph from df_test_set
  """
  if(un_graph == True):
    G = nx.from_pandas_edgelist(df, source = 'u', target = 'v', create_using = nx.Graph)
  else:
    G = nx.from_pandas_edgelist(df, source = 'u', target = 'v', create_using = nx.DiGraph)
  return G

def create_largest_component(df): #return graph, dataframe
  ## Create largest_component_graph (aka largest connected subgraph) ##
  
  # List of nodes of the largest_connected_subgraph (aka largest_component)
  largest_component_nodes = list(max(nx.connected_components(create_graph(df)), key=len)) 
  # List of edges of the largest_connected_subgraph (aka largest_component) from df
  largest_component_edges =  df[df['u'].isin(largest_component_nodes) & df['v'].isin(largest_component_nodes)]
  # Create largest_connected_subgraph 
  largest_component_graph = create_graph(largest_component_edges)
  return largest_component_graph, largest_component_edges

def get_selected_graph(G, G_edges, number_of_nodes, un_graph = True):
  random.seed(1)
  node_list = random.sample(G.nodes, number_of_nodes)

  edge_list = G_edges[G_edges['u'].isin(node_list) & G_edges['v'].isin(node_list)]

  if(un_graph == True):
    selected_G = nx.from_pandas_edgelist(edge_list, source = 'u', target = 'v', create_using = nx.Graph)
  else:
    selected_G = nx.from_pandas_edgelist(edge_list, source = 'u', target = 'v', create_using = nx.DiGraph)

  return selected_G, edge_list

def get_nonEdge_and_test(selected_G, original_test_set, un_graph = True): # returns dataframe, dataframe
  ## Get nonEdge and test dataframe
  ## nonEdge: dataframe containing the non-exist edges at the training time.
  ## test: dataframe containing the edge in nonEdge. exist = 1 for exist in df_test_set, 0 for not exist in df_test_set 

  # Get only edges exisiting in G from original_test_set
  test_set = original_test_set[original_test_set['u'].isin(selected_G.nodes) & original_test_set['v'].isin(selected_G.nodes)]
  # Get the original non_edge
  non_edge = pd.DataFrame(list(nx.non_edges(selected_G)), columns=['u', 'v'])
  
  test_set = pd.merge(non_edge, test_set, on=['u','v'], how='left', indicator='exist')
  test_set['exist'] = np.where(test_set['exist'] == 'both', 1, 0)
  return test_set

In [0]:
G = create_graph(org_train)
print("Is G connected? " + str(nx.is_connected(G)))

Is G connected? False


In [0]:
randomG, randomG_edges = get_selected_graph(G, org_train, 100000, un_graph = True)
print(nx.info(randomG))
print("Is randomG connected? " + str(nx.is_connected(randomG)))
del G
del randomG

Name: 
Type: Graph
Number of nodes: 10807
Number of edges: 9473
Average degree:   1.7531
Is randomG connected? False


In [0]:
subG_lc, subG_lc_edges = create_largest_component(randomG_edges)
#del org_train
print("Information of the largest component ")
print(nx.info(subG_lc))

Information of the largest component 
Name: 
Type: Graph
Number of nodes: 5650
Number of edges: 6560
Average degree:   2.3221


In [0]:
nx.is_connected(subG_lc)

True

In [0]:
validate = get_nonEdge_and_test(subG_lc, org_test)
#del org_test

In [0]:
validate['exist'].value_counts()[1]

179

In [0]:
#cn = nx.common_neighbors(subG_lc, validate[['u','v']].values.tolist())
#validate['cn'] = [score for u, v, score in cn]
aa = nx.adamic_adar_index(subG_lc, validate[['u','v']].values.tolist())
validate['aa'] = [score for u, v, score in aa]
pa = nx.preferential_attachment(subG_lc, validate[['u','v']].values.tolist())
validate['pa'] = [score for u, v, score in pa]
jc = nx.jaccard_coefficient(subG_lc, validate[['u','v']].values.tolist())
validate['jc'] = [score for u, v, score in jc]

In [0]:
validate['exist'].value_counts()[1]

135

In [0]:
def get_auc(validate):
  topN = validate['exist'].value_counts()[1]
  pred = [1]*topN +[0]*validate['exist'].value_counts()[0]
  
  # Common neighbours
  #validate = validate.sort_values(by='cn', ascending=False)
  #validate['pred_cn'] = pred
  #fpr, tpr, _ = metrics.roc_curve(validate['exist'], validate['pred_cn'], pos_label=1)
  #auc_cn = metrics.auc(fpr, tpr)

  # Adamic Adar
  validate = validate.sort_values(by='aa', ascending=False)
  validate['pred_aa'] = pred
  fpr, tpr, _ = metrics.roc_curve(validate['exist'], validate['pred_aa'], pos_label=1)
  auc_aa = metrics.auc(fpr, tpr)
  # Preferential attachment
  validate = validate.sort_values(by='pa', ascending=False)
  validate['pred_pa'] = pred
  fpr, tpr, _ = metrics.roc_curve(validate['exist'], validate['pred_pa'], pos_label=1)
  auc_pa = metrics.auc(fpr, tpr)
  # Jaccard Coefficient
  validate = validate.sort_values(by='jc', ascending=False)
  validate['pred_jc'] = pred
  fpr, tpr, _ = metrics.roc_curve(validate['exist'], validate['pred_jc'], pos_label=1)
  auc_jc = metrics.auc(fpr, tpr)
  return validate, auc_aa, auc_pa, auc_jc

In [0]:
validate['jc'].value_counts()
# report on those problems that I found
# confusion matrix

0.000000    15583348
1.000000      209822
0.500000       70776
0.333333       28680
0.250000        9812
              ...   
0.006452           1
0.027397           1
0.002924           1
0.015228           1
0.026667           1
Name: jc, Length: 381, dtype: int64

In [0]:
validate, auc_aa, auc_pa, auc_jc = get_auc(validate)

In [0]:
print(auc_aa, auc_pa, auc_jc)

0.5074032385805779 0.5148107086771107 0.4999957684840453


In [0]:
 GCC = nx.transitivity(G)
 print(GCC)


In [0]:
 ASP = nx.average_shortest_path_length(G)
 print(ASP)

6.279193781058764


In [0]:
density = nx.density(G)
print(density)

0.000514392532759868


In [0]:
# 
cn_acc = metrics.accuracy_score(validate['exist'], validate['pred_cn']) 
aa_acc = metrics.accuracy_score(validate['exist'], validate['pred_aa']) 
jc_acc = metrics.accuracy_score(validate['exist'], validate['pred_jc']) 
pa_acc = metrics.accuracy_score(validate['exist'], validate['pred_pa']) 

# Confusion matrix
cn_confusion = pd.crosstab(validate['exist'], validate['pred_cn'], rownames=['Actual'], colnames=['cn_Predicted'], margins=True)
aa_confusion = pd.crosstab(validate['exist'], validate['pred_aa'], rownames=['Actual'], colnames=['aa_Predicted'], margins=True)
jc_confusion = pd.crosstab(validate['exist'], validate['pred_jc'], rownames=['Actual'], colnames=['jc_Predicted'], margins=True)
pa_confusion = pd.crosstab(validate['exist'], validate['pred_pa'], rownames=['Actual'], colnames=['pa_Predicted'], margins=True)

In [0]:
print(aa_acc)
aa_confusion

0.9999833248338047


aa_Predicted,0,1,All
Actual,,,
0,15951597,133,15951730
1,133,2,135
All,15951730,135,15951865


In [0]:
print(jc_acc)
jc_confusion

0.9999830740794258


jc_Predicted,0,1,All
Actual,,,
0,15951595,135,15951730
1,135,0,135
All,15951730,135,15951865


In [0]:
print(pa_acc)
pa_confusion

0.9999835755881836


pa_Predicted,0,1,All
Actual,,,
0,15951599,131,15951730
1,131,4,135
All,15951730,135,15951865
